Window functions operate on a group of rows, referred to as a window, and calculate a return value for each row based on the group of rows. Window functions are useful for processing tasks such as calculating a moving average, computing a cumulative statistic, or accessing the value of rows given the relative position of the current row.

### Syntax

window_function [ nulls_option ] OVER
( [  { PARTITION | DISTRIBUTE } BY partition_col_name = partition_col_val ( [ , ... ] ) ]
  { ORDER | SORT } BY expression [ ASC | DESC ] [ NULLS { FIRST | LAST } ] [ , ... ]
  [ window_frame ] )


In [ ]:
from pyspark.sql import SparkSession

In [21]:
spark = SparkSession.builder \
    .appName("WindowFunctionsDemo") \
    .getOrCreate()

data = [
    ("Lisa", "Sales", 10000, 35),
    ("Evan", "Sales", 32000, 38),
    ("Fred", "Engineering", 21000, 28),
    ("Alex", "Sales", 30000, 33),
    ("Tom", "Engineering", 23000, 33),
    ("Jane", "Marketing", 29000, 28),
    ("Jeff", "Marketing", 35000, 38),
    ("Paul", "Engineering", 29000, 23),
    ("Chloe", "Engineering", 23000, 25),
    ("Helen", "Marketing", 29000, 40)
]

columns = ["name", "dept", "salary", "age"]
employees_df = spark.createDataFrame(data, columns)
employees_df.createOrReplaceTempView("employees")
spark.sql("SELECT * FROM employees").show()


+-----+-----------+------+---+
| name|       dept|salary|age|
+-----+-----------+------+---+
| Lisa|      Sales| 10000| 35|
| Evan|      Sales| 32000| 38|
| Fred|Engineering| 21000| 28|
| Alex|      Sales| 30000| 33|
|  Tom|Engineering| 23000| 33|
| Jane|  Marketing| 29000| 28|
| Jeff|  Marketing| 35000| 38|
| Paul|Engineering| 29000| 23|
|Chloe|Engineering| 23000| 25|
|Helen|  Marketing| 29000| 40|
+-----+-----------+------+---+



### Ranking Functions

RANK(): Assigns rank within the partition, with gaps if there are ties.

In [41]:
spark.sql("""
    SELECT 
       name, 
       dept, 
       salary, 
       RANK() OVER (PARTITION BY dept ORDER BY salary) AS rank 
    FROM employees
""").show()


+-----+-----------+------+----+
| name|       dept|salary|rank|
+-----+-----------+------+----+
| Fred|Engineering| 21000|   1|
|  Tom|Engineering| 23000|   2|
|Chloe|Engineering| 23000|   2|
| Paul|Engineering| 29000|   4|
| Jane|  Marketing| 29000|   1|
|Helen|  Marketing| 29000|   1|
| Jeff|  Marketing| 35000|   3|
| Lisa|      Sales| 10000|   1|
| Alex|      Sales| 30000|   2|
| Evan|      Sales| 32000|   3|
+-----+-----------+------+----+



DENSE_RANK(): Assigns rank within the partition without gaps for ties.

In [23]:
spark.sql("""
    SELECT 
       name, 
       dept, 
       salary,
       DENSE_RANK() OVER (PARTITION BY dept ORDER BY salary) AS dense_rank
    FROM employees
""").show()


+-----+-----------+------+----------+
| name|       dept|salary|dense_rank|
+-----+-----------+------+----------+
| Fred|Engineering| 21000|         1|
|  Tom|Engineering| 23000|         2|
|Chloe|Engineering| 23000|         2|
| Paul|Engineering| 29000|         3|
| Jane|  Marketing| 29000|         1|
|Helen|  Marketing| 29000|         1|
| Jeff|  Marketing| 35000|         2|
| Lisa|      Sales| 10000|         1|
| Alex|      Sales| 30000|         2|
| Evan|      Sales| 32000|         3|
+-----+-----------+------+----------+



PERCENT_RANK() calculates the relative rank of a row within a partition:

$$
\text{percent_rank} = \frac{\text{rank} - 1}{\text{rows in partition} - 1}
$$

In [29]:
spark.sql("""
    SELECT
       name,
       dept,
       salary,
       PERCENT_RANK() OVER (PARTITION BY dept ORDER BY salary) AS percent_rank
    FROM employees
""").show()

+-----+-----------+------+------------------+
| name|       dept|salary|      percent_rank|
+-----+-----------+------+------------------+
| Fred|Engineering| 21000|               0.0|
|  Tom|Engineering| 23000|0.3333333333333333|
|Chloe|Engineering| 23000|0.3333333333333333|
| Paul|Engineering| 29000|               1.0|
| Jane|  Marketing| 29000|               0.0|
|Helen|  Marketing| 29000|               0.0|
| Jeff|  Marketing| 35000|               1.0|
| Lisa|      Sales| 10000|               0.0|
| Alex|      Sales| 30000|               0.5|
| Evan|      Sales| 32000|               1.0|
+-----+-----------+------+------------------+



NTILE(N) divides the rows in each partition into N buckets of more or less equal size, and assigns each row to one of those buckets

In [32]:
spark.sql("""
    SELECT
       name,
       dept,
       salary,
       NTILE(3) OVER (PARTITION BY dept ORDER BY salary) AS salary_bucket
    FROM employees
""").show()

+-----+-----------+------+-------------+
| name|       dept|salary|salary_bucket|
+-----+-----------+------+-------------+
| Fred|Engineering| 21000|            1|
|  Tom|Engineering| 23000|            1|
|Chloe|Engineering| 23000|            2|
| Paul|Engineering| 29000|            3|
| Jane|  Marketing| 29000|            1|
|Helen|  Marketing| 29000|            2|
| Jeff|  Marketing| 35000|            3|
| Lisa|      Sales| 10000|            1|
| Alex|      Sales| 30000|            2|
| Evan|      Sales| 32000|            3|
+-----+-----------+------+-------------+



ROW_NUMBER() assigns an incrementing integer to each row within a partition, ordered by the specified column

In [33]:
spark.sql("""
    SELECT
       name,
       dept,
       salary,
       ROW_NUMBER() OVER (PARTITION BY dept ORDER BY salary) AS row_num
    FROM employees
""").show()


+-----+-----------+------+-------+
| name|       dept|salary|row_num|
+-----+-----------+------+-------+
| Fred|Engineering| 21000|      1|
|  Tom|Engineering| 23000|      2|
|Chloe|Engineering| 23000|      3|
| Paul|Engineering| 29000|      4|
| Jane|  Marketing| 29000|      1|
|Helen|  Marketing| 29000|      2|
| Jeff|  Marketing| 35000|      3|
| Lisa|      Sales| 10000|      1|
| Alex|      Sales| 30000|      2|
| Evan|      Sales| 32000|      3|
+-----+-----------+------+-------+



### Analytic Functions

Syntax for analytical functions are CUME_DIST(), LAG(), LEAD(), NTH_VALUE(), FIRST_VALUE(), LAST_VALUE()

In [37]:
spark.sql("""
    SELECT
       name,
       dept,
       age,
       CUME_DIST() OVER (PARTITION BY dept ORDER BY age) AS cume_dist
    FROM employees
""").show()


+-----+-----------+---+------------------+
| name|       dept|age|         cume_dist|
+-----+-----------+---+------------------+
| Paul|Engineering| 23|              0.25|
|Chloe|Engineering| 25|               0.5|
| Fred|Engineering| 28|              0.75|
|  Tom|Engineering| 33|               1.0|
| Jane|  Marketing| 28|0.3333333333333333|
| Jeff|  Marketing| 38|0.6666666666666666|
|Helen|  Marketing| 40|               1.0|
| Alex|      Sales| 33|0.3333333333333333|
| Lisa|      Sales| 35|0.6666666666666666|
| Evan|      Sales| 38|               1.0|
+-----+-----------+---+------------------+



In [26]:
spark.sql("""
    SELECT
       name,
       dept,
       salary,
       LAG(salary) OVER (PARTITION BY dept ORDER BY salary) AS lag,
       LEAD(salary, 1, 0) OVER (PARTITION BY dept ORDER BY salary) AS lead
    FROM employees
""").show()


+-----+-----------+------+-----+-----+
| name|       dept|salary|  lag| lead|
+-----+-----------+------+-----+-----+
| Fred|Engineering| 21000| NULL|23000|
|  Tom|Engineering| 23000|21000|23000|
|Chloe|Engineering| 23000|23000|29000|
| Paul|Engineering| 29000|23000|    0|
| Jane|  Marketing| 29000| NULL|29000|
|Helen|  Marketing| 29000|29000|35000|
| Jeff|  Marketing| 35000|29000|    0|
| Lisa|      Sales| 10000| NULL|30000|
| Alex|      Sales| 30000|10000|32000|
| Evan|      Sales| 32000|30000|    0|
+-----+-----------+------+-----+-----+



In [38]:
spark.sql("""
    SELECT
        name,
        dept,
        salary,
        
        FIRST_VALUE(salary) OVER (
            PARTITION BY dept
            ORDER BY salary
            ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
        ) AS first_salary,
        
        LAST_VALUE(salary) OVER (
            PARTITION BY dept
            ORDER BY salary
            ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
        ) AS last_salary,
        
        NTH_VALUE(salary, 2) OVER (
            PARTITION BY dept
            ORDER BY salary
            ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
        ) AS second_salary
        
    FROM employees
    ORDER BY dept, salary
""").show()

+-----+-----------+------+------------+-----------+-------------+
| name|       dept|salary|first_salary|last_salary|second_salary|
+-----+-----------+------+------------+-----------+-------------+
| Fred|Engineering| 21000|       21000|      29000|        23000|
|  Tom|Engineering| 23000|       21000|      29000|        23000|
|Chloe|Engineering| 23000|       21000|      29000|        23000|
| Paul|Engineering| 29000|       21000|      29000|        23000|
| Jane|  Marketing| 29000|       29000|      35000|        29000|
|Helen|  Marketing| 29000|       29000|      35000|        29000|
| Jeff|  Marketing| 35000|       29000|      35000|        29000|
| Lisa|      Sales| 10000|       10000|      32000|        30000|
| Alex|      Sales| 30000|       10000|      32000|        30000|
| Evan|      Sales| 32000|       10000|      32000|        30000|
+-----+-----------+------+------------+-----------+-------------+



### Aggregate Functions

There are different aggregation functions like COUNT(), SUM(), MIN(), MAX(), AVG()

In [36]:
spark.sql("""
    SELECT
       name,
       dept,
       salary,
       MIN(salary) OVER (PARTITION BY dept ORDER BY salary) AS min
    FROM employees
""").show()

+-----+-----------+------+-----+
| name|       dept|salary|  min|
+-----+-----------+------+-----+
| Fred|Engineering| 21000|21000|
|  Tom|Engineering| 23000|21000|
|Chloe|Engineering| 23000|21000|
| Paul|Engineering| 29000|21000|
| Jane|  Marketing| 29000|29000|
|Helen|  Marketing| 29000|29000|
| Jeff|  Marketing| 35000|29000|
| Lisa|      Sales| 10000|10000|
| Alex|      Sales| 30000|10000|
| Evan|      Sales| 32000|10000|
+-----+-----------+------+-----+

